<a href="https://colab.research.google.com/github/longtran22/NMKHDL_nhom2_IT4930/blob/main/CNN_Emotion_Regcognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
0!pip install kagglehub


Của Nguyễn Thế Nhật Minh data ảnh

In [ ]:
import kagglehub
import os
# Download latest version
path = kagglehub.dataset_download("akaiinu/crema-d")

print("Path to dataset files:", path)


Using Colab cache for faster access to the 'crema-d' dataset.
Path to dataset files: /kaggle/input/crema-d


In [ ]:
print("Số lượng file:", len(os.listdir(path+'/cremad/Image-01-FPS')))


Số lượng file: 7442


gán cặp

In [ ]:
import os, glob, pandas as pd

root_dir = "/root/.cache/kagglehub/datasets/akaiinu/crema-d/versions/1/cremad/Image-01-FPS"
rows = []
print(os.listdir(root_dir))
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    if not os.path.isdir(folder_path):
        continue

    # tách nhãn cảm xúc từ tên folder
    parts = folder.split("_")
    if len(parts) < 3:
        continue
    emotion = parts[2]  # ví dụ: ANG, HAP, SAD, NEU, FEA, DIS...

    # lấy tất cả ảnh trong folder đó
    for img_path in glob.glob(os.path.join(folder_path, "*.jpg")):
        rows.append([img_path, emotion])

df = pd.DataFrame(rows, columns=["image_path", "emotion"])
print("Số lượng ảnh:", len(df))
print("Các nhãn:", sorted(df["emotion"].unique()))
df.head()


['1006_TAI_NEU_XX', '1088_DFA_NEU_XX', '1089_TSI_FEA_XX', '1036_TSI_DIS_XX', '1053_IEO_HAP_LO', '1070_IWL_HAP_XX', '1056_TSI_FEA_XX', '1023_ITS_DIS_XX', '1040_IOM_NEU_XX', '1078_WSI_SAD_XX', '1027_IWW_FEA_XX', '1011_TIE_NEU_XX', '1075_TIE_HAP_XX', '1049_IEO_ANG_MD', '1014_DFA_NEU_XX', '1082_IEO_HAP_LO', '1086_ITH_FEA_XX', '1038_ITS_FEA_XX', '1082_DFA_FEA_XX', '1012_IWW_HAP_XX', '1019_IEO_SAD_HI', '1031_IEO_HAP_LO', '1043_MTI_ANG_XX', '1079_IOM_ANG_XX', '1010_ITH_FEA_XX', '1072_IEO_DIS_MD', '1019_WSI_ANG_XX', '1027_IEO_ANG_MD', '1086_TAI_NEU_XX', '1016_TIE_FEA_XX', '1005_IWL_DIS_XX', '1053_IOM_FEA_XX', '1033_TAI_DIS_XX', '1052_TSI_HAP_XX', '1030_TIE_HAP_XX', '1035_TSI_DIS_XX', '1028_IEO_HAP_MD', '1073_IWL_FEA_XX', '1027_IWW_DIS_XX', '1054_IEO_SAD_LO', '1018_TSI_DIS_XX', '1041_IEO_DIS_MD', '1076_TSI_FEA_XX', '1056_IEO_SAD_MD', '1044_IEO_SAD_MD', '1005_DFA_ANG_XX', '1034_IOM_ANG_XX', '1051_TIE_NEU_XX', '1068_IWL_ANG_XX', '1032_IEO_HAP_HI', '1030_IEO_HAP_HI', '1001_IEO_ANG_LO', '1051_TAI_H

,image_path,emotion
0,/root/.cache/kagglehub/datasets/akaiinu/crema-...,NEU
1,/root/.cache/kagglehub/datasets/akaiinu/crema-...,NEU
2,/root/.cache/kagglehub/datasets/akaiinu/crema-...,NEU
3,/root/.cache/kagglehub/datasets/akaiinu/crema-...,NEU
4,/root/.cache/kagglehub/datasets/akaiinu/crema-...,NEU


Encode nó

In [ ]:
emotions = sorted(df["emotion"].unique())
emo2idx = {emo: i for i, emo in enumerate(emotions)}
df["label"] = df["emotion"].map(emo2idx)
print(emo2idx)


{'ANG': 0, 'DIS': 1, 'FEA': 2, 'HAP': 3, 'NEU': 4, 'SAD': 5}


In [ ]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

class EmotionDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        img_path, label = self.df.iloc[idx][["image_path", "label"]]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, label


In [ ]:
!pip install cite

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.15, stratify=df["label"], random_state=42)
train_loader = DataLoader(EmotionDataset(train_df, transform), batch_size=64, shuffle=True)
val_loader   = DataLoader(EmotionDataset(val_df, transform), batch_size=64)


train

In [ ]:
!pip install torch tdqm

In [ ]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F

class SpatialAttention(nn.Module):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size=7, padding=3, bias=False)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        y = torch.cat([avg_out, max_out], dim=1)
        y = self.sigmoid(self.conv(y))
        return x * y

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()

        self.sa2 = SpatialAttention()
        self.sa3 = SpatialAttention()
        self.sa4 = SpatialAttention()
        # Block 1: 224x224 -> 112x112
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32) # Thêm Batch Norm
        self.pool = nn.MaxPool2d(2, 2)

        # Block 2: 112x112 -> 56x56
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.sa2 = SpatialAttention()

        # Block 3: 56x56 -> 28x28
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.sa3 = SpatialAttention()

        # Block 4: 28x28 -> 14x14
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.sa4 = SpatialAttention()

        # Block 5: 14x14 -> 7x7 (Thêm 1 block nữa để giảm kích thước ảnh)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)

        # Sau 5 lần pool: 224 -> 112 -> 56 -> 28 -> 14 -> 7
        self.flatten_size = 512 * 7 * 7

        self.fc1 = nn.Linear(self.flatten_size, 1024)
        self.bn_fc1 = nn.BatchNorm1d(1024) # Batch norm cho FC layer
        self.dropout = nn.Dropout(0.4)
        self.fc2 = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.sa2(x)
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.sa3(x)
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.sa4(x)
        x = self.pool(F.relu(self.bn5(self.conv5(x))))

        x = x.view(-1, self.flatten_size)

        x = F.relu(self.bn_fc1(self.fc1(x)))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


In [ ]:
# ==========================
# KHỞI TẠO SIMPLE CNN
# ==========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(emotions) # Đã có từ code cũ của bạn

model = SimpleCNN(num_classes=num_classes).to(device)

print(model) # In cấu trúc mạng ra để kiểm tra

# Thiết lập Loss và Optimizer
criterion = nn.CrossEntropyLoss()
# Vì train from scratch, ta thường dùng learning rate lớn hơn fine-tune một chút (ví dụ 1e-3 hoặc 3e-4)
optimizer = optim.Adam(model.parameters(), lr=3e-4)


In [ ]:
from tqdm.notebook import tqdm

# Số lượng epochs tùy bạn chọn (ví dụ 20 vì train from scratch lâu hội tụ hơn transfer learning)
epochs = 20

for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")

    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Tính accuracy train ngay trong loop nếu thích
        pred = out.argmax(1)
        correct += (pred == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item())

    train_acc = correct / total
    avg_train_loss = total_loss / len(train_loader)

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    val_loss = 0

    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            loss = criterion(out, labels)
            val_loss += loss.item()

            pred = out.argmax(1)
            val_correct += (pred == labels).sum().item()
            val_total += labels.size(0)

    print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, Train Acc={train_acc:.4f} | Val Loss={val_loss/len(val_loader):.4f}, Val Acc={val_correct/val_total:.4f}")

Epoch 1/20:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 1: Train Loss=1.7792, Train Acc=0.2455 | Val Loss=1.6923, Val Acc=0.2861


Epoch 2/20:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 2: Train Loss=1.5281, Train Acc=0.3779 | Val Loss=1.4545, Val Acc=0.4146


Epoch 3/20:   0%|          | 0/310 [00:00<?, ?it/s]

...

Epoch 18/20:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 18: Train Loss=0.1338, Train Acc=0.9533 | Val Loss=1.8764, Val Acc=0.5931


Epoch 19/20:   0%|          | 0/310 [00:00<?, ?it/s]

Epoch 19: Train Loss=0.1160, Train Acc=0.9617 | Val Loss=1.5598, Val Acc=0.6378


Epoch 20/20:   0%|          | 0/310 [00:00<?, ?it/s]